In [42]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as u

import mdtraj
import numpy as np

import time

## System

In [16]:
prmtop = app.AmberPrmtopFile('complex.top')
pdb = app.PDBFile('complex.pdb')
system = prmtop.createSystem(nonbondedMethod=app.PME, 
                             constraints=app.HBonds, 
                             nonbondedCutoff=12*u.angstroms, 
                             switchDistance=10*u.angstroms)
topology = mdtraj.Topology.from_openmm(prmtop.topology)

total_steps = 100000 # Reducing for testing purposes 3000000

## Integrator, forces and restraints

In [17]:
integrator = mm.LangevinIntegrator(50*u.kelvin, 5/u.picosecond, 0.002*u.picoseconds)

In [18]:
barostat = mm.MonteCarloBarostat(1.0*u.bar, 300*u.kelvin)

## Init simulation and positions

In [19]:
simulation = app.Simulation(prmtop.topology, system, integrator)

simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(50*u.kelvin)

## Minimize energy
100 steps while gradually releasing the constraints then
1000 steps with no constraints

In [20]:
simulation.minimizeEnergy(maxIterations=1000)

## Heating

In [21]:
for temperature in np.linspace(50, 300, 251)*u.kelvin:
    integrator.setTemperature(temperature)
    simulation.step(100)
    
simulation.step(5000)

## Equilibrate

In [29]:
simulation.system.addForce(barostat)

Exception: the System object does not own its corresponding OpenMM object

In [30]:
simulation.step(int(total_steps*1/3))

simulation.saveState('equilibrated.xml')

## Production

In [46]:
simulation.reporters.append(app.CheckpointReporter('checkpnt.chk', int(total_steps/10)))
simulation.reporters.append(app.DCDReporter('simulation.dcd', int(total_steps/400)))
simulation.reporters.append(app.StateDataReporter('simulation.log', int(total_steps/200) ,
                                              step=True, 
                                              time=True, 
                                              totalEnergy=True))

In [39]:
simulation.step(int(total_steps*2/3))
print()

In [44]:
print('Benchmarking...')
initial_time = time.time()

final_time = time.time()
elapsed_time = (final_time - initial_time) * u.seconds
simulated_time = total_steps * integrator.getStepSize()
performance = (simulated_time / elapsed_time)
print('completed %8d steps in %8.3f s : performance is %8.3f ns/day' % (total_steps, elapsed_time / u.seconds, performance / (u.nanoseconds/u.day)))

Benchmarking...
completed   100000 steps in    0.000 s : performance is 233798622.968 ns/day


In [47]:
print(time.time())

1510920335.190455
